In [34]:
from pytrends.request import TrendReq
import datetime

# Login to Google. Only need to run this once, the rest of requests will use the same session.
pytrend = TrendReq()

In [35]:
def getWeeks():

    dStart = input("Enter date of start epidemiological week in DD-MM-YYYY format: ")
    dStart = [int(i) for i in dStart.split("-")]
    start = datetime.datetime(dStart[2],dStart[1],dStart[0])
    
    dEnd = input("Enter date of end epidemiological week in DD-MM-YYYY format: ")
    dEnd = [int(i) for i in dEnd.split("-")]
    end = datetime.datetime(dEnd[2],dEnd[1],dEnd[0])
    
    nextWeek = getNextWeek(start)
    weeks = [start]
    
    while nextWeek <= end:
        if(nextWeek < datetime.datetime.today()):
            weeks.append(nextWeek)
            nextWeek = getNextWeek(nextWeek)
    
    return weeks

In [36]:
def getNextWeek(timeframe):
    return timeframe+datetime.timedelta(days=7)

In [37]:
def getYMD(timeframe):
    return "{}/{}/{}".format(timeframe.day,timeframe.month,timeframe.year)

In [38]:
year = input("Enter year: ")

Enter year: 2018


In [39]:
countryCode = input("Enter country code: ")

Enter country code: MX


In [40]:
weeks = getWeeks()

Enter date of start epidemiological week in DD-MM-YYYY format: 06-01-2018
Enter date of end epidemiological week in DD-MM-YYYY format: 05-05-2018


In [41]:
with open("{}/google/{}.csv".format(year, countryCode), "w") as file:
    weekEpiArray = []
    cities = {}
    file.write("CITY,")
    file.write(",".join(map(getYMD, weeks))+"\n")
    for i in range(len(weeks)):
        week = weeks[i]
        nextWeek = getNextWeek(week)
        year = week.year
        month = week.month
        day = week.day
        nextWeekC = nextWeek - datetime.timedelta(days=1)
        timeframe = "{}-{}-{} {}-{}-{}".format(year,month,day,nextWeekC.year,nextWeekC.month,nextWeekC.day)
        print("{}-{}-{} -> {}-{}-{}".format(year,month,day,nextWeekC.year,nextWeekC.month,nextWeekC.day))
        pytrend.build_payload(kw_list=['zika'], geo=countryCode, timeframe=timeframe)
        results = pytrend.interest_by_region(resolution="CITY")
        # if a city's search index is 0 it doesn't appear on results, so we have to keep track of it
        tempWeekArray = {}
        for index, row in results.iterrows():
            tempWeekArray[index] = row["zika"]
            if index not in cities:
                cities[index] = index #Store in dictionary to get O(1) access
        weekEpiArray.append(tempWeekArray)
        print("DONE", week)
        
    # Iterate on weekEpiArray and replace non existent cities with zeroes
    searchesByCity = {}
    for city in cities:
        searchesByCity[city] = []
        for week in weekEpiArray:
            if city not in week:
                searchesByCity[city].append(0)
            else:
                searchesByCity[city].append(week[city])
                
    # Finally write to file
    for city in sorted(searchesByCity.keys()):
        file.write("{},".format(city))
        file.write(",".join(map(str,searchesByCity[city])))
        file.write("\n")

2018-1-6 -> 2018-1-12
DONE 2018-01-06 00:00:00
2018-1-13 -> 2018-1-19
DONE 2018-01-13 00:00:00
2018-1-20 -> 2018-1-26
DONE 2018-01-20 00:00:00
2018-1-27 -> 2018-2-2
DONE 2018-01-27 00:00:00
2018-2-3 -> 2018-2-9
DONE 2018-02-03 00:00:00
2018-2-10 -> 2018-2-16
DONE 2018-02-10 00:00:00
2018-2-17 -> 2018-2-23
DONE 2018-02-17 00:00:00
2018-2-24 -> 2018-3-2
DONE 2018-02-24 00:00:00
2018-3-3 -> 2018-3-9
DONE 2018-03-03 00:00:00
2018-3-10 -> 2018-3-16
DONE 2018-03-10 00:00:00
2018-3-17 -> 2018-3-23
DONE 2018-03-17 00:00:00
2018-3-24 -> 2018-3-30
DONE 2018-03-24 00:00:00
2018-3-31 -> 2018-4-6
DONE 2018-03-31 00:00:00
2018-4-7 -> 2018-4-13
DONE 2018-04-07 00:00:00
2018-4-14 -> 2018-4-20
DONE 2018-04-14 00:00:00
2018-4-21 -> 2018-4-27
DONE 2018-04-21 00:00:00
2018-4-28 -> 2018-5-4
DONE 2018-04-28 00:00:00
2018-5-5 -> 2018-5-11
DONE 2018-05-05 00:00:00
